In [1]:
# Setup
import pandas as pd
import numpy as np

#Parameter Lists
#Set the variable as None to ignore it when searching
#Change the value here and run all the cells to get an output
LOCATION = None
ROOM = None
QUARTER_INDEX = None          # A number, refer to the file or dataframe
QUARTER = None                # e.g. "FA13"/"WI19"/"SP15"
COURSE_ID = "MATH"             # e.g. "COGS", "BILD"
COURSE_NUM = None  # Must be a list of string (double quoted or single quoted)! e.g. ["100", "20A", "120A"]
SECTION = None               # Must be a string! e.g "767325"
INSTRUCTOR_FIRST = None    # instructor's first name
INSTRUCTOR_LAST = None     # instructor's last name
LOWEST_SIZE = None            # Include only sections that have student number >= this variable
LOWEST_RESPONSE = None        # Include only Capes Questions that have number of response >= this variable
CAPES = [5,6,7,8,9,10,17,27,26]        # Must be a list of integer! Include only Capes question with an ID that is in the list. From 5 to 27, plus 51
FILENAME = "MATH_CAPES.csv"    # name of output file

ENABLE_TRANSPOSE = False

In [2]:
# Read CSV
dtypes = {'LOCATION':str, 'ROOM':str, 'QUARTER_INDEX':int, 'QUATER':int, 'COURSE_ID':str, 'COURSE_NUM':str, 
          'SECTION':str, 'INSTRUCTOR_LAST':str, 'INSTRUCTOR_FIRST': str, 'CAPE_ID':int, 'QUESTION':str, 
          'SECTION_SIZE':int, 'RESPONSE_SIZE':int, "RATING":float}
df = pd.read_csv("CAPES.csv", usecols = ["LOCATION", "ROOM", "QUARTER_INDEX", "QUARTER", 
                                         "COURSE_ID", "COURSE_NUM", "SECTION", "INSTRUCTOR_LAST", 
                                         "INSTRUCTOR_FIRST", "CAPE_ID", "QUESTION", 
                                         "SECTION_SIZE", "RESPONSE_SIZE", "RATING"],
                 dtype = dtypes)

# clean out empty spaces in cells
df["INSTRUCTOR_FIRST"] = df["INSTRUCTOR_FIRST"].str.strip()
df["INSTRUCTOR_LAST"] = df["INSTRUCTOR_LAST"].str.strip()

def ClassExtractor(LOCATION, ROOM, QUARTER, COURSE_ID, COURSE_NUM, SECTION, INSTRUCTOR_LAST, 
                   INSTRUCTOR_FIRST, LOWEST_SIZE, LOWEST_RESPONSE, CAPES):
    result = df
    
    if LOCATION is not None:
        result = result[result["LOCATION"] == LOCATION]
    
    if ROOM is not None:
        reulst = result[result["ROOM"] == ROOM]
        
    if QUARTER is not None:
        result = result[result["QUATER"] == QUARTER]
    
    if COURSE_ID is not None:
        result = result[result["COURSE_ID"] == COURSE_ID]
        
    if COURSE_NUM is not None:
        result = result[result["COURSE_NUM"].isin(COURSE_NUM)]
        
    if SECTION is not None:
        result = result[result["SECTION"] == SECTION]
    
    if INSTRUCTOR_LAST is not None:
        result = result[result["INSTRUCTOR_LAST"] == INSTRUCTOR_LAST]
        
    if INSTRUCTOR_FIRST is not None:
        result = result[result["INSTRUCTOR_FIRST"] == INSTRUCTOR_FIRST]
        
    if LOWEST_SIZE is not None:
        result = result[result["SECTION_SIZE"] >= LOWEST_SIZE]
        
    if LOWEST_RESPONSE is not None:
        result = result[result["RESPONSE_SIZE"] >= LOWEST_RESPONSE]
        
    if isinstance(CAPES, list):
        result = result[result["CAPE_ID"].isin(CAPES)]
    
    
    return result

def AlltimeAvg(INSTRUCTOR_FIRST, INSTRUCTOR_LAST, CAPES):
    newdf = (df.loc[df["INSTRUCTOR_LAST"] == INSTRUCTOR_LAST]).loc[df["INSTRUCTOR_FIRST"] == INSTRUCTOR_FIRST]
    dic = {}

    for i in CAPES:
        temp = newdf.loc[newdf["CAPE_ID"] == i]
        dic[i] = temp["RATING"].mean()

    print("All Time Average Rating for", INSTRUCTOR_LAST + ",", INSTRUCTOR_FIRST, "in (CAPE_ID, AVG)")
    for key, value in dic.items():
        print(key, value)

        
aggregations = {
    'RATING':'mean',
    'SECTION_SIZE':'sum',
    'RESPONSE_SIZE':'sum'
}
def transpose(output):
    hall = output
    data = hall.groupby(['CAPE_ID','SECTION']).agg(aggregations).reset_index()

    know_rating = pd.Series(data[data.CAPE_ID == 5].RATING)
    prep_rating = pd.Series(data[data.CAPE_ID == 6].RATING)
    audible_rating = pd.Series(data[data.CAPE_ID == 7].RATING)
    expl_rating = pd.Series(data[data.CAPE_ID == 8].RATING)
    attent_rating = pd.Series(data[data.CAPE_ID == 9].RATING)
    note_rating = pd.Series(data[data.CAPE_ID == 10].RATING)
    stimu_rating = pd.Series(data[data.CAPE_ID == 17].RATING)
    rec_course_rating = pd.Series(data[data.CAPE_ID == 26].RATING)
    rec_professor_rating = pd.Series(data[data.CAPE_ID == 27].RATING)
    df = hall
    df = df.drop_duplicates(subset=['SECTION'], keep='first')

    df.loc[:,'know_rating'] = know_rating.values
    df.loc[:,'prep_rating'] = prep_rating.values
    df.loc[:,'audible_rating'] = audible_rating.values
    df.loc[:,'expl_rating'] = expl_rating.values
    df.loc[:,'attent_rating'] = attent_rating.values
    df.loc[:,'note_rating'] = note_rating.values
    df.loc[:,'stimu_rating'] = stimu_rating.values
    df.loc[:,'rec_course_rating'] = rec_course_rating.values
    df.loc[:,'rec_professor_rating'] = rec_professor_rating.values
    df = df.drop(columns=['QUESTION','CAPE_ID'])

    
    return df

In [5]:
# Calling Function and Write Data
output = ClassExtractor(LOCATION, ROOM, QUARTER, COURSE_ID, COURSE_NUM, SECTION, INSTRUCTOR_LAST, 
                   INSTRUCTOR_FIRST, LOWEST_SIZE, LOWEST_RESPONSE, CAPES)
if ENABLE_TRANSPOSE:
    output = transpose(output)
    
filename = FILENAME or "CAPE_" + (INSTRUCTOR_FIRST or "") + " " + (INSTRUCTOR_LAST or "") + ".csv"
filename = 'data/' + filename

if not output.empty:
    output.to_csv(filename)

# if None in (INSTRUCTOR_FIRST, INSTRUCTOR_LAST):
#     INSTRUCTOR_FIRST = output["INSTRUCTOR_FIRST"].iloc[0]
#     INSTRUCTOR_LAST = output["INSTRUCTOR_LAST"].iloc[0]

# if CAPES is None:
#     CAPES = list(range(5, 28))
#     CAPES.append(51)
    
# AlltimeAvg(INSTRUCTOR_FIRST, INSTRUCTOR_LAST, CAPES)
# An summary of overall capes rating of this professor will be display below after running this cell